In [50]:
import numpy as np
from scipy.optimize import minimize_scalar
from typing import List, Tuple

In [51]:
# Dữ liệu đầu vào
A = np.array([[1, 2],
              [-1, 3],
              [0.5, -2]])
b = np.array([0.1, 0.2, -0.1])

In [52]:
# Cài đặt hàm f(x) đề bài yêu cầu và gradient của hàm f(x)
def f(x: np.ndarray) -> float:
    return np.log(np.sum(np.exp(A @ x + b)))

def grad_f(x: np.ndarray) -> np.ndarray:
    exps = np.exp(A @ x + b)
    return (exps @ A) / np.sum(exps)


In [53]:
# Cài đặt thuật toán theo phương pháp Parabol
def parabolic_interpolation(phi, t0 = 0, t1 = 1e-2, t2=1e-1, tol=1e-5, max_iter=100):
    for _ in range(max_iter):
        y0, y1, y2 = phi(t0), phi(t1), phi(t2)
        numerator = (t1 - t0)**2 * (y1 - y2) - (t1 - t2)**2 * (y1 - y0)
        demoniator = (t1 - t0) * (y1 - y2) - (t1 - t2) * (y1 - y2)
        if demoniator == 0:
            break
        t_min = t1 - 0.5 * numerator / demoniator
        if abs(t_min - t1) < tol:
            return t_min
        
        t0, t1, t2 = t1, t2, t_min
    
    return t1

def gradient_descent_parabolic(x0, epsilon=1e-5, max_iter=100):
    x = x0.copy()
    history = [x.copy()]
    for _ in range(max_iter):
        g = grad_f(x)
        if np.linalg.norm(g) < epsilon:
            break
        phi = lambda t: f(x - t * g)
        t = parabolic_interpolation(phi)
        x = x - t * g
        history.append(x.copy())

    return x, history

In [54]:
# Cài đặt thuật toán theo phương pháp Brent
def gradient_descent_brent(x0, epsilon=1e-5, max_iter=100):
    x = x0.copy()
    history = [x.copy()]
    for _ in range(max_iter):
        g = grad_f(x)
        if np.linalg.norm(g) < epsilon:
            break
        phi = lambda t: f(x - t * g)
        res = minimize_scalar(phi, method='brent')
        t = res.x
        x = x - t * g
        history.append(x.copy())

    return x, history

In [55]:
x0 = np.array([0.0, 0.0])

x_parab, hist_prab = gradient_descent_parabolic(x0)
x_brent, hist_brent = gradient_descent_brent(x0)

print(f"Nghiệm gần đúng theo phương pháp Parabol: {x_parab}. Số vòng lặp theo phương pháp Parabol: {len(hist_prab) - 1}")
print(f"Nghiệm gần đúng theo phương pháp Brent: {x_brent}. Số vòng lặp theo phương pháp Brent: {len(hist_brent) - 1}")


Nghiệm gần đúng theo phương pháp Parabol: [-0.6842572  -0.38376046]. Số vòng lặp theo phương pháp Parabol: 100
Nghiệm gần đúng theo phương pháp Brent: [-1.09048829 -0.48931366]. Số vòng lặp theo phương pháp Brent: 87
